# Perfect Virtual- 
Cryptocurrency usecase: Utilise a form  of Recurrent Neural Network (RNN) model for the prediction of Bitcoin prices from historical data and analysis of social media trends. 
## Build Process:
1. Generate API key to access Bitcoin data from the [cryptocompare API](https://min-api.cryptocompare.com/)
2. Choose relevent data features e.g. price, volume, online engagement
3. Preprocess the data
4. Build the model architecture 
5. Train the model
6. Evaluate model performance
7. Identify feature importance

## Load the necessary libraries and packages:

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import keras 
from keras.models import Sequential
from keras import layers
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import cryptocompare
import requests
import datetime
# pretty printing of pandas dataframe
pd.set_option('expand_frame_repr', False) 

# Retrieve historical Bitcoin  price data:
Define the api key for access to data from Cryptocompare API 

In [ ]:
apiKey = "98901418980628c0e095f5d511be7435c1cbf090ed813f6c82fc88a8f939c9a3"

## Define function for retrieving json data from the API:

In [ ]:
def daily_price_historical(symbol, comparison_symbol, all_data=True, limit=1, aggregate=1, exchange=''):
    url = 'https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&limit={}&aggregate={}'\
        .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)
    if exchange:
        url += '&e={}'.format(exchange)
    if all_data:
        url += '&allData=true'
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
    return df

In [ ]:
df = daily_price_historical('BTC','USD')
df = df.drop('time', 1)
df.head()

## Define function to plot data:

In [ ]:
def plot_data(df, cryptocurrency, target_currency):
    plt.figure(figsize=(15,5))
    plt.title('{} / {} price data'.format(cryptocurrency, target_currency))
    plt.plot(df.timestamp, df.close, label ='close')
    plt.legend()
    plt.show()
    
    return None

In [ ]:
cryptocurrency = 'BTC'
target_currency = 'USD'
plot_data(df, cryptocurrency, target_currency)

# Retrieve historical Bitcoin social stats data:

## Create LSTM model for Bitcoin price prediction:

In [ ]:
df = df.drop('conversionType', 1)
df = df.drop('conversionSymbol',1)
df.head()

In [ ]:
df.isnull().values.any()

In [ ]:
data_training = df[df['timestamp']< '2020-01-01'].copy()
data_training

In [ ]:
data_test = df[df['timestamp']< '2020-01-01'].copy()
data_test

In [ ]:
training_data = data_training.drop(['timestamp'], axis = 1)
training_data.head()

In [ ]:
scaler = MinMaxScaler()
training_data = scaler.fit_transform(training_data)
training_data

In [ ]:
training_data.shape[0]

In [ ]:
X_train = [] 
Y_train = []

for i in range(60, training_data.shape[0]):
    X_train.append(training_data[i-60:i])
    Y_train.append(training_data[i,0])
X_train, Y_train = np.array(X_train), np.array(Y_train)
X_train.shape

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
#Initialize the RNN
model = Sequential() 
model.add(LSTM(units = 50, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], 6)))
model.add(Dropout(0.2)) 
model.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
model.add(Dropout(0.3)) 
model.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
model.add(Dropout(0.4)) 
model.add(LSTM(units = 120, activation = 'relu'))
model.add(Dropout(0.5)) 
model.add(Dense(units =1))
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
history= model.fit(X_train, Y_train, epochs = 20, batch_size =50, validation_split=0.1)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

In [ ]:
part_60_days = data_training.tail(60)
df_1= part_60_days.append(data_test, ignore_index = True)
df_1 = df_1.drop(['timestamp'], axis = 1)
df_1.head()

In [ ]:
inputs = scaler.transform(df_1) 
#inputs

In [ ]:
X_test = []
Y_test = []
for i in range (60, inputs.shape[0]):
    X_test.append(inputs[i-60:i]) 
    Y_test.append(inputs[i, 0])
X_test, Y_test = np.array(X_test), np.array(Y_test) 
#X_test.shape, Y_test.shape
Y_pred = model.predict(X_test) 
#Y_pred, Y_test
scaler.scale_

In [ ]:
scale = 1/5.18164146e-05
Y_test = Y_test*scale 
Y_pred = Y_pred*scale
Y_pred

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(Y_test, color = 'red', label = 'Real Bitcoin Price')
plt.plot(Y_pred, color = 'green', label = 'Predicted Bitcoin Price')
plt.title('Bitcoin Price Prediction (LSTM)')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()